<a href="https://colab.research.google.com/github/Jefferson-Luis/Data-Science-com-Python/blob/main/Modulo_5_Modelagem_Classifica%C3%A7%C3%A3o_exercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercícios:

Proposta:

* Refazer o pipeline completo, para Gradient_boosting aproveitando os códigos já desenvolvidos no notebook, usando todas as variáveis no processo de modelagem

* Comparar os resultados com os anteriores

*Em tuning de hiperparametros: n_estimators: [500, 1000]; min_samples_leaf[2,100];
min_samples_plist:[10,200]; random_state:[42]

** Passos a serem seguidos:**
*  Selecionar os dados de origem
*  Quebrar as categoricas e numericas
*  Tratar as variáveis categoricas.
*  Juntar os dados e formar a ABT
*  Aplicar feature selection
*  Selecionar os dados para ABT final de modelagem
*  Estruturar o dicionário par atuning
*  Instânciar os obj
*  Executar o trains
*  Observar os resultados

In [ ]:
# Manipulação de dados

import pandas as pd

# Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Quebra dos dados de train / test
from sklearn.model_selection import train_test_split

# Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# Modelos de classificação 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Vaidação cruzada
from sklearn.model_selection import cross_val_score

# Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

# Tuning de hiperparâmetros
from sklearn.model_selection import GridSearchCV


# adicionar algumas configurações de visualizações

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
metadados = pd.read_excel('metadata.xlsx')

In [ ]:
df = pd.read_csv('new_train.csv', sep=',')

In [ ]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,999,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,999,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,999,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,999,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,999,0,nonexistent,no


In [ ]:
df_exercicio = df.copy()

In [ ]:
df_exercicio.dtypes

age             int64
job            object
marital        object
education      object
default        object
housing        object
loan           object
contact        object
month          object
day_of_week    object
duration        int64
campaign        int64
pdays           int64
previous        int64
poutcome       object
y              object
dtype: object

Separando numericas das categoricas

In [ ]:
# categoricas
df_exercicio_cat = df_exercicio.drop(columns=['age','duration','campaign','pdays','previous','y'], axis=1)

In [ ]:
df_exercicio_cat.columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'day_of_week', 'poutcome'],
      dtype='object')

In [ ]:
df_exercicio_num = df_exercicio.drop(columns=['job','marital','education','default','housing','loan','contact',
       'month','day_of_week','poutcome','y'])

In [ ]:
df_exercicio_num.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')

In [ ]:
df_exercicio_cat_num = pd.get_dummies(df_exercicio_cat,
                                      prefix_sep= '_',
                                      columns=df_exercicio_cat.columns,
                                      drop_first=True)

In [ ]:
df_exercicio_cat_num.columns

Index(['job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'marital_unknown',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_unknown',
       'default_yes', 'housing_unknown', 'housing_yes', 'loan_unknown',
       'loan_yes', 'contact_telephone', 'month_aug', 'month_dec', 'month_jul',
       'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct',
       'month_sep', 'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
df_exercicio_cat_num.head()

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0


In [ ]:
df_exercicio_feature_selection = df_exercicio_cat_num.merge(df_exercicio_num, left_index=True, right_index=True)

In [ ]:
df_exercicio_feature_selection.shape

(32950, 48)

In [ ]:
df_exercicio_var_resp = df_exercicio['y']

In [ ]:
df_exercicio_var_resp.head()

0     no
1     no
2    yes
3     no
4     no
Name: y, dtype: object

In [ ]:
df_exercicio_var_resp = df_exercicio_var_resp.replace('no',0)
df_exercicio_var_resp = df_exercicio_var_resp.replace('yes',1)

In [ ]:
df_exercicio_var_resp.head()

0    0
1    0
2    1
3    0
4    0
Name: y, dtype: int64

seleção de variáveis

In [ ]:
# categoricas


df_exercicio_cat_dum_norm = MinMaxScaler().fit_transform(df_exercicio_cat_num)
selecao_chi = SelectKBest(chi2, k=5)
selecao_chi.fit(df_exercicio_cat_dum_norm, df_exercicio_var_resp)

suporte_chi = selecao_chi.get_support()
features_chi = df_exercicio_cat_num.loc[:,suporte_chi].columns.tolist()
features_chi 


['contact_telephone',
 'month_mar',
 'month_oct',
 'month_sep',
 'poutcome_success']

In [ ]:
# numericas

selecao_rfe = RFE(estimator=LogisticRegression(random_state=42, max_iter=200), n_features_to_select=3, step=1)
selecao_rfe.fit(df_exercicio_num, df_exercicio_var_resp)

rfe_suporte = selecao_rfe.get_support()
rfe_features = df_exercicio_num.loc[:,rfe_suporte].columns.tolist()
rfe_features

['age', 'campaign', 'previous']

In [ ]:

df_exercicio_modelagem = df_exercicio_cat_num.merge(df_exercicio_num, left_index=True, right_index=True)

In [ ]:
x_treino_exer, x_teste_exer, y_treino_exer, y_teste_exer = train_test_split(df_exercicio_modelagem,
                                                                            df_exercicio_var_resp,
                                                                            test_size=0.3,
                                                                            random_state=42)

In [ ]:
x_treino_exer.head()

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,marital_unknown,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,age,duration,campaign,pdays,previous
24186,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,33,39,3,999,0
18087,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,48,90,2,999,0
6950,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,31,114,3,999,0
8200,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,40,138,3,999,0
16051,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,30,252,1,999,0


In [ ]:
# se  não estiver já realizado o import, então:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
exercicio_dict = {
    'n_estimators':[500,1000], 
    'min_samples_leaf':[2,100],
    'min_samples_split':[10,200],
    'random_state':[42]


}

In [ ]:
# modelo

gb_exercicio = GradientBoostingClassifier(random_state=42)



In [ ]:
# tuning

gb_exercicio_tuning = GridSearchCV(gb_exercicio,
                                   exercicio_dict,
                                   cv=2,
                                   scoring=make_scorer(accuracy_score))

In [ ]:
gb_exercicio_tuning.fit(x_treino_exer, y_treino_exer)

GridSearchCV(cv=2, estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'min_samples_leaf': [2, 100],
                         'min_samples_split': [10, 200],
                         'n_estimators': [500, 1000], 'random_state': [42]},
             scoring=make_scorer(accuracy_score))

In [ ]:
gb_exercicio_tuning.best_estimator_ 

GradientBoostingClassifier(min_samples_leaf=100, min_samples_split=10,
                           n_estimators=500, random_state=42)

In [ ]:
gb_exercicio_tuning.best_score_

0.9081291754776646

In [ ]:
accuracy_score(y_treino_exer, gb_exercicio_tuning.predict(x_treino_exer))

0.9213960546282246

In [ ]:
accuracy_score(y_teste_exer, gb_exercicio_tuning.predict(x_teste_exer))

0.9062215477996965

In [ ]:
import joblib

In [ ]:
joblib.dump(gb_exercicio_tuning, 'gb_exercicio_tuning.pkl')

['gb_exercicio_tuning.pkl']

In [ ]:
gb_grid_dc = {
    'n_estimators':[100,200,500],
    'min_samples_split':[2,40],
    'min_samples_leaf':[2,100],
    'random_state':[42]
}
gb_grid_estimator = GradientBoostingClassifier()

In [ ]:
gb_grid_acc= GridSearchCV(gb_grid_estimator,
                       gb_grid_dc,
                       scoring=make_scorer(accuracy_score),
                       cv=2)

In [ ]:
# gb_grid_acc.fit(x_treino, y_treino)
# accuracy_score(y_treino, gb_grid_acc.predict(x_treino))
# accuracy_score(y_teste, gb_grid_acc.predict(x_teste))